* What linguistic features might be influencing the classifier's decisions:  

1. The average Kband of the transcript - The more popular the talk, the lower the Kband mean.
    Why? Because we usually don't want the speaker to use a lot of terminologies that are only used in the field they explain or work.  

2. _need to think about one more feature that might affect the classifier's decision.._

* How to define _popular_ talk?  
Mainly `comments` column since it has higher level of engagement than `view`.
But I am planning to figure out how to incorporate the `view` information too

In [2]:
import pandas as pd
import numpy as np
import nltk
import math

In [3]:
# read files
ted_main = pd.read_csv("../data/ted_main.csv")
tran = pd.read_csv("../data/transcripts.csv")

In [4]:
# remove NAs
ted_main.dropna(inplace=True)
tran.dropna(inplace=True)

In [5]:
ted_main.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [6]:
# sorting the main TED conference 
ted = ted_main[ted_main['event'].str.fullmatch(r'TED\d\d\d\d')]
ted.event.unique()

array(['TED2006', 'TED2004', 'TED2005', 'TED2003', 'TED2007', 'TED2002',
       'TED2008', 'TED1984', 'TED1990', 'TED1998', 'TED2001', 'TED2009',
       'TED2010', 'TED2011', 'TED1994', 'TED2012', 'TED2013', 'TED2014',
       'TED2015', 'TED2016', 'TED2017'], dtype=object)

In [7]:
# combine two files
ted = pd.merge(ted_main, tran, on = 'url')
ted.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre..."
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac..."


In [22]:
# select the columns I need
ted_clean = ted[['title', 'event', 'comments', 'transcript']]
ted_clean.head()

,title,event,comments,transcript
0,Do schools kill creativity?,TED2006,4553,Good morning. How are you?(Laughter)It's been ...
1,Averting the climate crisis,TED2006,265,"Thank you so much, Chris. And it's truly a gre..."
2,Simplicity sells,TED2006,124,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,Greening the ghetto,TED2006,200,If you're here today — and I'm very happy that...
4,The best stats you've ever seen,TED2006,593,"About 10 years ago, I took on the task to teac..."


In [9]:
ted_clean.describe()

,comments
count,2461.000000
mean,192.722471
std,285.141388
min,2.000000
25%,63.000000
50%,117.000000
75%,223.000000
max,6404.000000


In [26]:
# tokenize the transcript
ted_clean['token'] = ted_clean.transcript.map(nltk.word_tokenize)

C:\Users\82102\AppData\Local\Temp\ipykernel_28864\1970647423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ted_clean['token'] = ted_clean.transcript.map(nltk.word_tokenize)


In [27]:
ted_clean.sample(6)

,title,event,comments,transcript,token
249,What happens when you lose everything,TED2008,166,I had a fire nine days ago. My archive: 175 fi...,"[I, had, a, fire, nine, days, ago, ., My, arch..."
1168,The attitudes that sparked Arab Spring,TEDxSummit,79,My talk today is about something maybe a coupl...,"[My, talk, today, is, about, something, maybe,..."
887,Turning trash into toys for learning,INK Conference,119,"My name is Arvind Gupta, and I'm a toymaker. I...","[My, name, is, Arvind, Gupta, ,, and, I, 'm, a..."
756,"Poverty, money -- and love",TEDGlobal 2010,718,The stories we tell about each other matter ve...,"[The, stories, we, tell, about, each, other, m..."
1311,A bath without water,TED@Johannesburg,129,"So I grew up in Limpopo, on the border of Limp...","[So, I, grew, up, in, Limpopo, ,, on, the, bor..."
6,Letting go of God,TED2006,919,"On September 10, the morning of my seventh bir...","[On, September, 10, ,, the, morning, of, my, s..."


In [12]:
# read files to make kband for the transcript analysis
flines = open("C://Users//82102//Desktop//program//2022 fall//LING 2231//count_1w.txt").readlines()
google_1gram_tup = [tuple(line.split()) for line in flines]
google_1gram_rank = {word:index+1 for (index,(word, count)) in enumerate(google_1gram_tup)}

In [13]:
# Build kband function
def get_avg_google_kband(toks):
    kbands = []
    for t in toks:
        if t.lower() in google_1gram_rank: 
           grank = google_1gram_rank[t.lower()]
           if grank <= 100000:  
               kband = math.floor(grank/1000)+1
               kbands.append(kband)
    return np.mean(kbands)

In [14]:
get_avg_google_kband(ted_clean.token[0])

2.712989957887917

In [28]:
ted_clean['kband_ave'] = ted_clean.token.map(get_avg_google_kband)

C:\Users\82102\AppData\Local\Temp\ipykernel_28864\888917555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ted_clean['kband_ave'] = ted_clean.token.map(get_avg_google_kband)


In [29]:
ted_clean.describe()

,comments,kband_ave
count,2461.000000,2461.000000
mean,192.722471,3.081130
std,285.141388,0.771869
min,2.000000,1.775387
25%,63.000000,2.638944
50%,117.000000,2.954208
75%,223.000000,3.376289
max,6404.000000,13.780000


* There are some unexpectedly high mean Kbands.. Let's see what they are.

In [17]:
# check the max kband_ave because it does not make sense..
ted_clean[ted_clean.kband_ave == 13.78]

,title,event,comments,transcript,token,kband_ave
97,"""La Vie en Rose""",TED2004,39,Thomas Dolby: For pure pleasure please welcome...,"[Thomas, Dolby, :, For, pure, pleasure, please...",13.78


In [18]:
# checking the token # to check if the token number is causing this problem
ted_clean[ted_clean.kband_ave == 13.78]['token'].map(len)

97    151
Name: token, dtype: int64

In [19]:
# check which talk has a kband mean higher than
ted_clean[ted_clean.kband_ave >= 5]['token'].map(len)

83      1314
85       185
97       151
146        9
272      583
385     1198
408     3294
574       53
580     1009
908       12
986      372
1170      24
1362      10
1465    1728
1623    2323
1792       6
1912     514
1913      12
1965    1593
1994      37
2062    1556
2418     420
Name: token, dtype: int64

* There are total 20 of them whose average kband is higher than 5. Let's check each one of them and see if we should remove them or not.  
* Maybe #1792 should be removed..  
* Or, is this the problem of the limit when I make kband? Should I narrow it down?.. Hmm..

In [31]:
ted_clean = ted_clean[ted_clean.kband_ave <= 5]

In [33]:
ted_clean.kband_ave.describe()

count    2439.000000
mean        3.039130
std         0.572755
min         1.775387
25%         2.637424
50%         2.950527
75%         3.359243
max         4.972222
Name: kband_ave, dtype: float64

* Looking alright!

**Build a classifier, using rating**